# CommandRemoteControl

Mise en oeuvre du design pattern "Command" en suivant l'exemple de la télécommande donné dans HFDP. The remote control has 7 slots that it can drive with on/off, there's also a "undo" button that we want to implement.

Intérêt du Pattern : chaque appareil que l'on veut commander peut être livré avec un API, que l'on connaît mais sur laquelle on n'a pas de maîtrise, et qui n'aurai pas forcément spécifiquement de bouton on/off. De plus on ne sait pas ce qu'on aura à contrôler dans le futur --> intéressant de pouvoir disposer d'une façon de contrôler tout ça. La télécommande ne doit à aucun moment savoir ce qu'elle contrôle.

In [1]:
# Import de annotation des méthodes abstraites
from __future__ import annotations
from abc import ABC, abstractmethod

In [2]:
class Command(ABC):
    
    def execute() -> None:
        pass
    
    def undo() -> None:
        pass

In [3]:
class Light():
    
    name: str = None
        
    def __init__(self, name:str):
        self.name = name
        
    def getName(self) -> str:
        return self.name
    
    def on(self) -> None:
        print(f'Light {self.name} is On')
        
    def off(self) -> None:
        print(f'Light {self.name} is Off')

In [4]:
class LightOnCommand(Command):
    
    light:Light = None
    
    def __init__(self, light:Light) -> None:
        self.light = light
    
    def execute(self):
        self.light.on()
        #print(f'Light {light.name} is On')
        
    def undo(self) -> None:
        self.light.off()

In [5]:
class LightOffCommand(Command):
    
    light:Light = None
    
    def __init__(self, light:Light) -> None:
        self.light = light
    
    def execute(self):
        self.light.off()
        
    def undo(self) -> None:
        self.light.on()

In [6]:
class SimpleRemoteControl():
    slot:Command = None
        
    def setCommand (self, command:Command) -> None:
        self.slot = command
        
    def buttonWasPressed (self) -> None:
        self.slot.execute()

### Code test

In [7]:
# On cree une telecommande
remote = SimpleRemoteControl()
# On instancie une lampe à contrôler
light = Light("kitchen")
# On instancie le contrôleur pour allumer la lampe
lightOn = LightOnCommand(light)
# On affecte la commande à la télécommande
remote.setCommand(lightOn)
remote.buttonWasPressed()

Light kitchen is On


In [8]:
# On instancie un contrôleur pour éteindre la lampe
lightOff = LightOffCommand(light)
# On réaffecte la commande de la télécommande
remote.setCommand(lightOff)
# On appuie sur le bouton
remote.buttonWasPressed()

Light kitchen is Off


### Autre exemple : lecteur audio

In [9]:
class Stereo():
    
    volume:int = None
    
    def on(self) -> None:
        print('Stereo is On')
        
    def off(self) -> None:
        print('Stereo is Off')
        
    def setCD(self) -> None:
        print('Stereo was set a CD')
        
    def setDVD(self) -> None:
        print('Stereo was set a DVD')
        
    def setVolume(self, volume:int) -> None:
        self.volume = volume
        print(f'Stereo Volume was set to {self.volume}')

In [10]:
class StereoOnWithCDCommand(Command):
    stereo:Stereo
        
    def __init__(self, stereo:Stereo):
        self.stereo = stereo
        
    def execute(self) -> None:
        self.stereo.on()
        self.stereo.setCD()
        self.stereo.setVolume(11)
        
    def undo(self) -> None:
        self.stereo.off()

In [11]:
class StereoOffCommand(Command):
    stereo:Stereo
        
    def __init__(self, stereo:Stereo):
        self.stereo = stereo
        
    def execute(self) -> None:
        self.stereo.off()
        
    def undo(self) -> None:
        self.stereo.on()
        self.stereo.setCD()
        self.stereo.setVolume(11)

In [12]:
# On crée une instance de stereo que l'on va contrôler
stereo = Stereo()
# On instancie la commande contrôlant la stereo
stereoOn = StereoOnWithCDCommand(stereo)
# On affecte la commande
remote.setCommand(stereoOn)
# On appuie sur le bouton
remote.buttonWasPressed()

Stereo is On
Stereo was set a CD
Stereo Volume was set to 11


### On ajoute d'autres devices à contrôler

In [13]:
class GarageDoor():
    
    def up(self):
        print("Opening garage door")
        
    def down(self):
        print("Closing garage door")
        
    def stop(self):
        print("Stop moving garage door")
        
    def lightOn(self):
        print("Garage light on")
        
    def lightOff(self):
        print("Garage light off")

In [14]:
class GarageDoorUpCommand(Command):
    garageDoor: GarageDoor = None
        
    def __init__(self, garageDoor:GarageDoor):
        self.garageDoor = garageDoor
        
    def execute(self) -> None:
        self.garageDoor.up()
        
    def undo(self) -> None:
        self.garageDoor.down()

In [15]:
class GarageDoorDownCommand(Command):
    garageDoor: GarageDoor = None
        
    def __init__(self, garageDoor:GarageDoor):
        self.garageDoor = garageDoor
        
    def execute(self) -> None:
        self.garageDoor.down()
    
    def undo(self) -> None:
        self.garageDoor.up()

In [16]:
class CeilingFan():
    
    speed:int = None
    
    def high(self):
        self.speed = 3
    
    def medium(self):
        self.speed = 2
        
    def low(self):
        self.speed = 1
        
    def off(self):
        self.speed = 0
        
    def getSpeed(self) -> int:
        return self.speed

In [17]:
class CeilingFanOffCommand(Command):
    fan: CeilingFan = None
    prevSpeed:int = None
        
    def __init__(self, fan:CeilingFan):
        self.fan = fan
        
    def execute(self) -> None:
        self.prevSpeed = self.fan.getSpeed()
        self.fan.off()
        print(f"Vitesse du ventilateur: {self.fan.getSpeed()}")
        
    def undo(self) -> None:
        self.fan.medium()
        print(f"Vitesse du ventilateur: {self.fan.getSpeed()}")

In [18]:
class CeilingFanHighCommand(Command):
    fan: CeilingFan = None
    prevSpeed:int = None
        
    def __init__(self, fan:CeilingFan):
        self.fan = fan
        
    def execute(self) -> None:
        self.prevSpeed = self.fan.getSpeed()
        self.fan.high()
        print(f"Vitesse du ventilateur: {self.fan.getSpeed()}")
        
    def undo(self) -> None:
        if self.prevSpeed == 3:
            self.fan.high()
        elif self.prevSpeed == 2:
            self.fan.medium()
        elif self.prevSpeed == 1:
            self.fan.low()
        else:
            self.fan.off()
        print(f"Vitesse du ventilateur: {self.fan.getSpeed()}")

In [19]:
class CeilingFanMediumCommand(Command):
    fan: CeilingFan = None
    prevSpeed:int = None
        
    def __init__(self, fan:CeilingFan):
        self.fan = fan
        
    def execute(self) -> None:
        prevSpeed = self.fan.getSpeed()
        self.fan.medium()
        print(f"Vitesse du ventilateur: {self.fan.getSpeed()}")
        
    def undo(self) -> None:
        if self.prevSpeed == 3:
            self.fan.high()
        elif self.prevSpeed == 2:
            self.fan.medium()
        elif self.prevSpeed == 1:
            self.fan.low()
        else:
            self.fan.off()
        print(f"Vitesse du ventilateur: {self.fan.getSpeed()}")

In [20]:
garage = GarageDoor()
garage.up()
garage.lightOff()

Opening garage door
Garage light off


## Télécommande complète

Pseudo Pattern Null Object : classe qui ne fait rien !

In [21]:
class NoCommand(Command):
    
    def execute(self) -> None:
        pass
    
    def undo(self) -> None:
        pass

In [22]:
class RemoteControl():
    
    # Liste de commandes On et Off
    onCommands: Command = []
    offCommands: Command = []
    # Commande Undo
    undoCommand: Command = None
    
    # Constructeur
    def __init__(self) -> None:
        # Initialisation avec un objet nul
        noCommand:Command = NoCommand()
        undoCommand = noCommand
        for i in range(7):
            self.onCommands.append(noCommand)
            self.offCommands.append(noCommand)
        
            
    # Affectation des commandes
    def setCommand(self, slot:int, onCommand:Command, offCommand:Command):
        self.onCommands[slot] = onCommand
        self.offCommands[slot] = offCommand
        
    def onButtonWasPushed(self, slot:int) -> None:
        self.onCommands[slot].execute()
        self.undoCommand = self.onCommands[slot]
        
    def offButtonWasPushed(self, slot:int) -> None:
        self.offCommands[slot].execute()
        self.undoCommand = self.offCommands[slot]
        
    def undoCommandWasPushed(self) -> None:
        self.undoCommand.undo()
        
    def __str__(self) -> str:
        tableConfig: str = []
        
        for i in range(len(self.onCommands)):
            tableConfig.append(f'[Slot {i}] {self.onCommands[i].__class__.__name__}    {self.offCommands[i].__class__.__name__} \n')
        
        return ''.join(tableConfig)

In [23]:
# Create the remote
remoteControl = RemoteControl()
# Create the equipments
livingRoomLight = Light('Living Room')
kitchenLight = Light('Kitchen')
fan = CeilingFan()
garageDoor = GarageDoor()
stereo = Stereo()
# Create the commands
livingRoomLightOn = LightOnCommand(livingRoomLight)
livingRoomLightOff = LightOffCommand(livingRoomLight)
kitchenLightOn = LightOnCommand(kitchenLight)
kitchenLightOff = LightOffCommand(kitchenLight)
ceilingFanMedium = CeilingFanMediumCommand(fan)
ceilingFanHigh = CeilingFanHighCommand(fan)
ceilingFanOff = CeilingFanOffCommand(fan)
garageDoorUp = GarageDoorUpCommand(garageDoor)
garageDoorDown = GarageDoorDownCommand(garageDoor)
stereoOnWithCD = StereoOnWithCDCommand(stereo)
stereoOff = StereoOffCommand(stereo)
# Set commands
remoteControl.setCommand(0,livingRoomLightOn, livingRoomLightOff)
remoteControl.setCommand(1,kitchenLightOn, kitchenLightOff)
remoteControl.setCommand(2, garageDoorUp, garageDoorDown)
remoteControl.setCommand(3, stereoOnWithCD, stereoOff)
remoteControl.setCommand(4, ceilingFanMedium, ceilingFanOff)
remoteControl.setCommand(5, ceilingFanHigh, ceilingFanOff)

In [25]:
print(remoteControl)

[Slot 0] LightOnCommand    LightOffCommand 
[Slot 1] LightOnCommand    LightOffCommand 
[Slot 2] GarageDoorUpCommand    GarageDoorDownCommand 
[Slot 3] StereoOnWithCDCommand    StereoOffCommand 
[Slot 4] CeilingFanMediumCommand    CeilingFanOffCommand 
[Slot 5] CeilingFanHighCommand    CeilingFanOffCommand 
[Slot 6] NoCommand    NoCommand 



In [26]:
remoteControl.onButtonWasPushed(0)
remoteControl.offButtonWasPushed(0)
remoteControl.onButtonWasPushed(1)
remoteControl.offButtonWasPushed(1)
remoteControl.onButtonWasPushed(2)
remoteControl.offButtonWasPushed(2)
remoteControl.onButtonWasPushed(3)
remoteControl.offButtonWasPushed(3)
remoteControl.onButtonWasPushed(4)
remoteControl.offButtonWasPushed(4)

Light Living Room is On
Light Living Room is Off
Light Kitchen is On
Light Kitchen is Off
Opening garage door
Closing garage door
Stereo is On
Stereo was set a CD
Stereo Volume was set to 11
Stereo is Off
Vitesse du ventilateur: 2
Vitesse du ventilateur: 0


### Test du bouton undo :

In [27]:
remoteControl.onButtonWasPushed(0)
remoteControl.offButtonWasPushed(0)
remoteControl.undoCommandWasPushed()
remoteControl.offButtonWasPushed(0)

Light Living Room is On
Light Living Room is Off
Light Living Room is On
Light Living Room is Off


### Test du undo pour le ventilateur

In [28]:
print("Allume moyen")
remoteControl.onButtonWasPushed(4)
print("Éteins moyen")
remoteControl.offButtonWasPushed(4)
print("Annule")
remoteControl.undoCommandWasPushed()
print("Allume haut")
remoteControl.onButtonWasPushed(5)
print("Annule")
remoteControl.undoCommandWasPushed()

Allume moyen
Vitesse du ventilateur: 2
Éteins moyen
Vitesse du ventilateur: 0
Annule
Vitesse du ventilateur: 2
Allume haut
Vitesse du ventilateur: 3
Annule
Vitesse du ventilateur: 2


## Macro command

Exécution de plusieurs commandes à la fois!

In [42]:
class MacroCommand(Command):
    commands: Command = []
    
    def __init__(self, commands: Command) -> None:
        self.commands = commands
        
    def execute(self) -> None:
        for command in self.commands:
            command.execute()
            
    def undo(self) -> None:
        for command in self.commands:
            command.undo()

In [43]:
arrivingHome = [livingRoomLightOn, kitchenLightOn, stereoOnWithCD, ceilingFanMedium, garageDoorUp]
leavingHome = [livingRoomLightOff, kitchenLightOff, stereoOff, ceilingFanOff, garageDoorDown]

In [44]:
arrivingHomeMacro = MacroCommand(arrivingHome)
leavingHomeMacro = MacroCommand(leavingHome)

In [45]:
remoteControl.setCommand(6, arrivingHomeMacro, leavingHomeMacro)

In [46]:
remoteControl.onButtonWasPushed(6)

Light Living Room is On
Light Kitchen is On
Stereo is On
Stereo was set a CD
Stereo Volume was set to 11
Vitesse du ventilateur: 2
Opening garage door


In [50]:
remoteControl.offButtonWasPushed(0)

Light Living Room is Off


In [51]:
remoteControl.undoCommandWasPushed()

Light Living Room is On


In [52]:
remoteControl.offButtonWasPushed(6)

Light Living Room is Off
Light Kitchen is Off
Stereo is Off
Vitesse du ventilateur: 0
Closing garage door


In [53]:
remoteControl.onButtonWasPushed(6)

Light Living Room is On
Light Kitchen is On
Stereo is On
Stereo was set a CD
Stereo Volume was set to 11
Vitesse du ventilateur: 2
Opening garage door


In [54]:
remoteControl.undoCommandWasPushed()

Light Living Room is Off
Light Kitchen is Off
Stereo is Off
Vitesse du ventilateur: 0
Closing garage door
